In [1]:
apikey = 'ce04ffa20c7ee4560e4ffd4761bbe157'

In [2]:
import requests
from bs4 import BeautifulSoup
import bs4
import json
import httpx
import time
import lxml

In [16]:
def search_for_papers(query = 'nips, membrane, polysulfone', apikey = 'ce04ffa20c7ee4560e4ffd4761bbe157', count=1, start=0, sort='relevance', view='COMPLETE'):
    url = 'https://api.elsevier.com/content/search/sciencedirect?'
    headers = {'X-ELS-APIKey': apikey, 'Accept': 'application/json'}
    params = {'query': query, 'count': count, 'start': start, 'sort': sort, 'view': view}
    r = requests.get(url, headers=headers, params=params)
    json_resp = json.loads(r.text)
    titles, piis, authors = [], [], []
    dic = {}
    print(r)
    print(json_resp)
    for paper in json_resp['search-results']['entry']:
        titles.append(paper['dc:title'])
        piis.append(paper['pii'])
        dic[paper['dc:title']] = paper['pii']
        authors.append(paper['ce:given-name'])  
    return dic

In [17]:
search_for_papers()

<Response [200]>
{'search-results': {'opensearch:totalResults': '853', 'opensearch:startIndex': '0', 'opensearch:itemsPerPage': '1', 'opensearch:Query': {'@role': 'request', '@searchTerms': 'nips, membrane, polysulfone', '@startPage': '0'}, 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=0&count=1&query=nips%2C+membrane%2C+polysulfone&sort=relevance&view=COMPLETE', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'first', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=0&count=1&query=nips%2C+membrane%2C+polysulfone&sort=relevance&view=COMPLETE', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'next', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=1&count=1&query=nips%2C+membrane%2C+polysulfone&sort=relevance&view=COMPLETE', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'last', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=852&count=

KeyError: 'ce:given-name'

In [12]:
def get_paragraphs(pii, apikey, timeout=60):
    url = f'https://api.elsevier.com/content/article/pii/{pii}'
    response = requests.get(url, headers={"X-ELS-APIKey":apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = timeout)
    soup = BeautifulSoup(response.content, "xml")
    text =[''.join(s.findAll(text=True)).replace("\u202f", "").replace("\n", "") for s in soup.find_all('ce:para')]
    #text =[''.join(s.findAll(text=True)) for s in soup.find_all('ce:section')]
    return soup

In [44]:
paper = get_paragraphs('S2213343722016402', apikey)
paper

<?xml version="1.0" encoding="utf-8"?>
<full-text-retrieval-response xmlns="http://www.elsevier.com/xml/svapi/article/dtd" xmlns:bk="http://www.elsevier.com/xml/bk/dtd" xmlns:cals="http://www.elsevier.com/xml/common/cals/dtd" xmlns:ce="http://www.elsevier.com/xml/common/dtd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:ja="http://www.elsevier.com/xml/ja/dtd" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:prism="http://prismstandard.org/namespaces/basic/2.0/" xmlns:sa="http://www.elsevier.com/xml/common/struct-aff/dtd" xmlns:sb="http://www.elsevier.com/xml/common/struct-bib/dtd" xmlns:tb="http://www.elsevier.com/xml/common/table/dtd" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xocs="http://www.elsevier.com/xml/xocs/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><coredata><prism:url>https://api.elsevier.com/content/article/pii/S2213343722016402</prism:url><dc:identifier>doi:10.1016/j.jece.2022.108767</dc:identifier><ei

In [62]:
[''.join(s.findAll(text=True)) for s in paper.find_all('sa:organization')]#[:2]


['School of Chemistry, College of Science, University of Tehran']

In [57]:
first_name = [''.join(s.findAll(text=True)) for s in paper.find_all('ce:given-name')][0]
last_name = [''.join(s.findAll(text=True)) for s in paper.find_all('ce:surname')][0]
name = first_name + ' ' + last_name

In [8]:
class paper_parser():
    def __init__(self, query, apikey, count, start, sort, view, timeout):
        self.query = query
        self.apikey = apikey
        self.count = count
        self.start = start
        self.sort = sort
        self.view = view
        self.timeout = timeout
        self.dic = search_for_papers(query, apikey, count, start, sort, view)
        self.pii = list(self.dic.values())
        self.paragraphs = []
    def search_for_papers(query, apikey, count, start, sort, view):
        url = 'https://api.elsevier.com/content/search/sciencedirect?'
        headers = {'X-ELS-APIKey': apikey, 'Accept': 'application/json'}
        params = {'query': query, 'count': count, 'start': start, 'sort': sort, 'view': view}
        r = requests.get(url, headers=headers, params=params)
        json_resp = json.loads(r.text)
        titles, piis = [], []
        dic = {}
        for paper in json_resp['search-results']['entry']:
            titles.append(paper['dc:title'])
            piis.append(paper['pii'])
            dic[paper['dc:title']] = paper['pii']
        return dic
    def get_paragraphs(pii, apikey, timeout):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = timeout)
        soup = BeautifulSoup(response.content, "xml")
        text =[''.join(s.findAll(text=True)).replace("\u202f", "").replace("\n", "") for s in soup.find_all('ce:para')]
        return text
    def get_all_paragraphs(self):
        for i in self.pii:
            self.paragraphs.append(get_paragraphs(i, self.apikey, self.timeout))
        return self.paragraphs

In [9]:
pp = paper_parser('nips, membrane, polysulfone', apikey, 1, 0, 'relevance', 'COMPLETE', 60)
pp.get_all_paragraphs()

<Response [200]>
{'search-results': {'opensearch:totalResults': '849', 'opensearch:startIndex': '0', 'opensearch:itemsPerPage': '1', 'opensearch:Query': {'@role': 'request', '@searchTerms': 'nips, membrane, polysulfone', '@startPage': '0'}, 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=0&count=1&query=nips%2C+membrane%2C+polysulfone&sort=relevance&view=COMPLETE', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'first', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=0&count=1&query=nips%2C+membrane%2C+polysulfone&sort=relevance&view=COMPLETE', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'next', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=1&count=1&query=nips%2C+membrane%2C+polysulfone&sort=relevance&view=COMPLETE', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'last', '@href': 'https://api.elsevier.com/content/search/sciencedirect?start=848&count=

[['Versatile, facile, easy, and economical, these are the advantages that immersion precipitation or non-solvent phase separation (NIPS) offers [1–3]. Due to these, NIPS process has been regarded as the most widely-used approach in fabricating flat sheet membranes from a variety of polymers [4]. In addition, NIPS can provide a wide-range of pore size: from microfiltration to nanofiltration [5]. In this process, a polymer of choice is dissolved in a suitable solvent. Once homogeneous, the polymeric solution is cast onto a support and phase-inversion is prompted through direct immersion in a coagulation bath/non-solvent bath (typically water), generating a polymer rich phase that transforms into the membrane matrix after polymer precipitation, and a polymer lean phase which develops into pores after demixing [6].',
  'Polysulfone (PSf) membrane is amongst the most extensively used commercial membranes due to its favorable attributes such as high glass transition temperature (Tg\xa0=\xa01